In [1]:
from __future__ import annotations

import os

import numpy as np
import pandas as pd
import torch
from PIL import Image, ImageOps
from datasets import load_metric
from tqdm.notebook import tqdm
from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor
)
from ultralytics import YOLO

In [2]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

In [3]:
class TextRecognizePipeline:
    
    def __init__(self):
        self.ocr_processor = TrOCRProcessor.from_pretrained("raxtemur/trocr-base-ru")
        # self.ocr_model = VisionEncoderDecoderModel.from_pretrained("../../models/text_recognizer/trocr_ru_pretrain_3epoch/", local_files_only=True).to(device)

        # self.ocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")
        self.ocr_model = VisionEncoderDecoderModel.from_pretrained("../../models/text_recognizer/checkpoint-1152/", local_files_only=True).to(device)
        self.ocr_model.eval()
        
        # self.detection_model = YOLO("../../models/new_text_detector/best.pt").to(device)
        self.detection_model = YOLO("/media/admin01/storage1/vadim/Historical-docs-OCR/models/text_detector/best_1024.pt").to(device)
        self.iou_threshold = 0.7
        
        # metrics
        self.iou_list = []
        self.cer_list = []
        self.wer_list = []
        
        # Set special tokens used for creating the decoder_input_ids from the labels.
        self.ocr_model.config.decoder_start_token_id = self.ocr_processor.tokenizer.cls_token_id
        self.ocr_model.config.pad_token_id = self.ocr_processor.tokenizer.pad_token_id
        # Set Correct vocab size.
        self.ocr_model.config.vocab_size = self.ocr_model.config.decoder.vocab_size
        self.ocr_model.config.eos_token_id = self.ocr_processor.tokenizer.sep_token_id
        
        self.ocr_model.config.max_length = 64
        self.ocr_model.config.early_stopping = True
        self.ocr_model.config.no_repeat_ngram_size = 3
        self.ocr_model.config.length_penalty = 2.0
        self.ocr_model.config.num_beams = 4
    
    def get_detections_and_crop_boxes(self, img: Image) -> list[Image]:
        
        def sort_bbox_by_y(bbox_list):
            sorted_bbox = sorted(bbox_list, key=lambda bbox: (bbox[1], bbox[0]))  # Сортировка по координате y, затем по x
            return sorted_bbox
        
        result = []
        for predict, image in zip(self.detection_model.predict([img], verbose=False), [img]):
            bboxes = predict.boxes.xyxy.cpu().tolist()
            sorted_bboxes = sort_bbox_by_y(bboxes)
            for box in sorted_bboxes:
                cropped_image = image.crop(box)
                result.append(cropped_image.convert("RGB"))
        return result
    
    def get_ocr_predictions(self, img_list: list[Image]) -> list[str]:
        with torch.no_grad():
            pixel_values = self.ocr_processor(img_list, return_tensors="pt").pixel_values.to(device)
            generated_ids = self.ocr_model.generate(pixel_values)
            generated_text = self.ocr_processor.batch_decode(generated_ids, skip_special_tokens=True)
            
        return generated_text
    
    def recognize(self, img_list: list[Image]) -> list[str]:
        cropped_images = self.get_detections_and_crop_boxes(img_list)
        recognized_text = self.get_ocr_predictions(cropped_images)
        return recognized_text

In [4]:
import pathlib

def get_rand_image():
    path = pathlib.Path("../../data/processed/3 Production/text_detector/test/images")
    img_path = np.random.choice(list(path.iterdir()))
    img = Image.open(img_path)
    img = ImageOps.exif_transpose(img)
    return img, img_path

def get_label_text(data: pd.DataFrame, filename: str) -> list[str]:
    return data[data["file_name"].str.contains(filename)]["text"].to_list()

def extract_filename(filename):
    base_name, extension = os.path.splitext(filename)
    parts = base_name.split("___")
    return parts[0] + extension

def get_image(img_path: str | pathlib.Path) -> Image:
    image = Image.open(img_path)
    image = ImageOps.exif_transpose(image)
    return image

In [5]:
data = pd.read_csv("../../data/processed/3 Production/test.csv", index_col=0)
# data['file_name'] = data['file_name'].apply(lambda x: x.replace('.JPG', '.jpg'))

# 0 - Губернаторские отчёты
# 1 - Уставные грамоты – Афанасенков
# 2 - Уставные грамоты в jpg (Просветов)
# 3 - Победоносцев

# отделяем губернаторские отчёты и уставные грамоты
governors_reports = data[data["label"] == 0]
charter_letters = data[(data["label"] == 1) | (data["label"] == 2)]
segment_annotation = data[data["label"] == 3]

In [6]:
data['label'].unique()

array([0, 1, 2, 3])

In [7]:
ocr_pipeline = TextRecognizePipeline()

cer_metric = load_metric("cer", trust_remote_code=True)
wer_metric = load_metric("wer", trust_remote_code=True)

/tmp/ipykernel_3904985/3705093783.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer", trust_remote_code=True)


#### Подсчёт CER/WER для губернаторских отчётов

In [8]:
governors_reports["file_name"] = governors_reports["file_name"].apply(extract_filename)
filenames = list(governors_reports.file_name.unique())

/tmp/ipykernel_3904985/2217891385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  governors_reports["file_name"] = governors_reports["file_name"].apply(extract_filename)


In [9]:
image_root_path = pathlib.Path("../../data/processed/3 Production/text_detector/test/images")

cer = []
wer = []

for filename in tqdm(filenames, total=len(filenames)):
    file_path = image_root_path / pathlib.Path(filename)
    
    if not os.path.exists(file_path):
        file_path = image_root_path / pathlib.Path(filename.replace('.JPG', '.jpg'))
        
    img = get_image(file_path)
        
    pred_text = ocr_pipeline.recognize(img)
    pred_text = " ".join(pred_text)
    
    label_text = get_label_text(governors_reports, filename)
    label_text = " ".join(label_text)
    
    cer.append(
        cer_metric.compute(predictions=[pred_text], 
                           references=[label_text])
    )
    
    wer.append(
        wer_metric.compute(predictions=[pred_text], 
                           references=[label_text])
    )

print(f"CER: {np.mean(cer)} | WER: {np.mean(wer)}")

  0%|          | 0/298 [00:00<?, ?it/s]

CER: 0.08646262894234258 | WER: 0.2559889583773403


#### Подсчёт CER/WER для отчётных грамот

In [10]:
charter_letters["file_name"] = charter_letters["file_name"].apply(extract_filename)
filenames = list(charter_letters.file_name.unique())

/tmp/ipykernel_3904985/1796841258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charter_letters["file_name"] = charter_letters["file_name"].apply(extract_filename)


In [11]:
image_root_path = pathlib.Path("../../data/processed/3 Production/text_detector/test/images")

cer = []
wer = []

for filename in tqdm(filenames, total=len(filenames)):
    file_path = image_root_path / pathlib.Path(filename)
    
    if not os.path.exists(file_path):
        file_path = image_root_path / pathlib.Path(filename.replace('.JPG', '.jpg'))
    
    img = get_image(file_path)
        
    pred_text = ocr_pipeline.recognize(img)
    pred_text = " ".join(pred_text)
    
    label_text = get_label_text(charter_letters, filename)
    label_text = " ".join(label_text)
    
    cer.append(
        cer_metric.compute(predictions=[pred_text], references=[label_text])
    )
    
    wer.append(
        wer_metric.compute(predictions=[pred_text], references=[label_text])
    )
    
print(f"CER: {np.mean(cer)} | WER: {np.mean(wer)}")

  0%|          | 0/59 [00:00<?, ?it/s]

CER: 0.12972285992238589 | WER: 0.37622438091415655


Подсчёт CER/WER для 'Победоносцев' (резметка сегментами)

In [12]:
segment_annotation["file_name"] = segment_annotation["file_name"].apply(extract_filename)
filenames = list(segment_annotation.file_name.unique())

/tmp/ipykernel_3904985/4162330369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segment_annotation["file_name"] = segment_annotation["file_name"].apply(extract_filename)


In [13]:
image_root_path = pathlib.Path("../../data/processed/3 Production/text_detector/test/images")

cer = []
wer = []

for filename in tqdm(filenames, total=len(filenames)):
    file_path = image_root_path / pathlib.Path(filename)
    img = get_image(file_path)
    
    pred_text = ocr_pipeline.recognize(img)
    pred_text = " ".join(pred_text)
    
    label_text = get_label_text(segment_annotation, filename)
    label_text = " ".join(label_text)
    
    cer.append(
        cer_metric.compute(predictions=[pred_text], references=[label_text])
    )
    
    wer.append(
        wer_metric.compute(predictions=[pred_text], references=[label_text])
    )

print(f"CER: {np.mean(cer)} | WER: {np.mean(wer)}")

  0%|          | 0/9 [00:00<?, ?it/s]

CER: 0.4731776838193206 | WER: 0.7276771050396693


In [23]:
file_path

PosixPath('../../data/processed/3 Production/text_detector/test/images/c6d63ae5-15.png')

In [21]:
label_text

'1866. Въ Декабре, 2 числа, началъ, по желанiю Императрицы, занятiя съ цесаревной рускою исторiей Преобладанiе Гр П. А. Шувалова. Графъ Паленъ – министръ юстицi. 1867. Апрель. Пасха въ Москве, съ Катеи. 6 Iюня. Едемъ на лето въ Гомель, съ Софьей Никоноровной, съ Соничкой и Володей. 17 авг. уезжаемъ обратно 28 августа – до 3 сент. я въ Москве, один. но вскоре уехалъ туда снова, съ Катей. + 6 Сентября скончалась милая маменька. Похоронили ее 9 числа на Ваганькове! 15 Сент. страшная смерть Чивилева въ Ц. Селе. Занятiя съ В. К. Владимiромъ Александровичемъ. 22 окт. свадьба племянницы – Машеньки асеевой. + 19 ноября. Митроп. Московскiй Филаретъ 24 Ноября.– 27. Ездилъ съ В. К. Владимiромъ въ Москву на отпеванiе митрополита. Занятiя съ цесаревичемъ остановились, по воз вращенiи его изъ заграницы. 1868. вечера у Кн. в. п. Мещерского съ цесаревичемъ. Исторiя о голоде въ арханг. губ. – Комитетъ. 26 Янв. возобновл. занятiя съ цесаревной. Мерз. Тимашевъ переселъ на место Валуева. Я назначенъ Сенат

In [20]:
pred_text

'Въ декабрь, 2 числа, ничасть, пожеланiю Императрицы, за съ цесаревной русскою исторчей Преоблюдате де П. А. Шувалова. Постъ Полянъ – Министръ юстицы. Апрель. Пана въ Москве, съкатею 6 Iюня: Вдень на поэтовъ Гансонъ, съ сажей Никоноровной съ соничкой и Еголовей. 17 авг. уезжаемъ обротила 28 августа – до 3 Сент. я въ Москве, одинъ, На вскоре уехалъ туда снова, съ Кате † 6 Сентября скончалась милая моненька Покорники ее 9 число на Ваганькове. 15 Сент. строенная Смерть Чивикеевъ въ ц. Силет Занятiя съ В. К. Владимира Александровичемъ. 22 окт. свадьба плетянинцы. Мошеньки сельскiй † 19 ноября. Минирал. московскiй фелоратъ 24 ноября. Уездалъ съ В. К. Владимиромъ 6 Москве, но по отзыванiе митрополита. Занятiя съ цесаревичемъ остановились, по во вращенiй его изъ за границы. вечера у Кн. в. П. Мещерскаго съ Цесаревичамъ Исторiя о голоде въ арханъ. губ. – Комитетъ 26 Iюня. возобновл. занятiя съ цесаревной Март. Тимошевъ потоколь на симъ Я Пораже Сенаторомъ и ничасы присутств вать во 22–ть 16 Ма

In [26]:
data.groupby('label')['label'].value_counts()

label
0    6101
1     720
2     595
3     326
Name: count, dtype: int64

#TODO: Добавить выгрузку по bbox распознавание + разметка

In [27]:
segment_annotation

,file_name,text,label
0,909ccbb0-18.png,Начало общества у В.К. Константина. Отделъ Общ...,3
1,909ccbb0-18.png,Валуевъ – М-ръ Госуд. имуществъ.,3
2,909ccbb0-18.png,20 мая. Поездка съ Катей и Соничкой черезъ Москву,3
3,909ccbb0-18.png,въ Смоленскъ. у а. в. шевандиной и у Дiодора,3
4,909ccbb0-18.png,въ Александровскомъ. вернулись 1 Iюня.,3
...,...,...,...
321,c6d63ae5-15.png,"Варшаву и Берлинъ, и Парижъ и Лондонъ,",3
322,c6d63ae5-15.png,на о-въ Вайтъ. – Шенклинъ. На обратномъ,3
323,c6d63ae5-15.png,Пути черезъ Ломжу – возвращаемся,3
324,c6d63ae5-15.png,1 Сентября.,3
